In [14]:
import json
import pandas as pd

In [ ]:

from util.tools import parse_inq_art_url

# Step 1: Load the JSON file
with open('news_articles.json', 'r') as f:
    data = json.load(f)

# Step 2: Flatten the structure
records = []
for category, dates in data.items():
    for date, urls in dates.items():
        for url in urls:
            records.append({
                'category': category,
                'date': date,
                'url': url
            })

# Step 3: Convert to DataFrame
df = pd.DataFrame(records)

# Optional: Convert date column to datetime type
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

df['id'] = df['url'].apply(lambda x: parse_inq_art_url(x)['subdomain'] + ':' + parse_inq_art_url(x)['article_id'] + ':' + parse_inq_art_url(x)['slug'])


In [19]:
from util.biquery import create_or_update_table

create_or_update_table(
    df,
    'ph_news',
    'articles'
)

INFO - Table `articles` in dataset `ph_news` updated with mode `append`.


In [28]:
df = pd.read_csv('article_content.csv')

print(df[df['date_time'].isna()]['url'].unique())

[]
